In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
import yfinance as yf
import bs4 as bs
import datetime

In [2]:
# Read and print the stock tickers that make up S&P500
response = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = bs.BeautifulSoup(response.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})

In [3]:
tickers = []
for row in table.findAll('tr')[1:]:
    ticker = row.findAll('td')[0].text
    tickers.append(ticker)
tickers = [s.replace('\n', '') for s in tickers]

In [4]:
#start_date = datetime.datetime(2023, 5, 1)
#end_date = datetime.datetime(2024, 5, 1)
#data = yf.download(tickers, start=start_date, end=end_date)
#data

In [5]:
#final_df.append(
#    pd.Series(
#        [
#            symbol,
#            price,
#            marketCap,
#            'N/A'
#        ],
#        index=df_columns
#    ),
#    ignore_index=True
#)

In [6]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


In [7]:
from secrets_1 import IEX_CLOUD_API_TOKEN

In [8]:
symbol = 'BRK-B'
#token = 'LZ2CUPOJF3FIIQ8Q'
#function = 'GLOBAL_QUOTE'
#api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
#api_url = f'https://www.alphavantage.co/query?function={function}&symbol={symbol}&interval=5min&apikey={token}'
#data = requests.get(api_url).json()
data = yf.Ticker(symbol)
#print(data.status_code)
print(data)

yfinance.Ticker object <BRK-B>


In [9]:
# Get the historical prices for this ticker
tickerDf = data.history(period='1d', start='2024-05-30', end='2024-05-31')
marketCap= data.info['marketCap']
price= tickerDf['Open'][0]

print(marketCap/1000000000000)
#price
#tickerDf


0.887574429696


In [10]:
df_columns = ['Ticker', 'Stock Price', 'Market Cap', '# of shares to buy']
sample_df = pd.DataFrame(columns = df_columns)
sample_df

,Ticker,Stock Price,Market Cap,# of shares to buy


In [11]:
sample_df.append(
    pd.Series(
        [
            symbol,
            price,
            marketCap,
            'N/A'
        ],
        index=df_columns
    ),
    ignore_index=True
)

,Ticker,Stock Price,Market Cap,# of shares to buy
0,BRK-B,403.769989,887574429696,N/A


In [12]:
marketCap

887574429696

In [20]:
# Loop thru every ticker in the csv
full_df = pd.DataFrame(columns=df_columns)
for symbol in tickers:
    symbol = symbol.replace(".", "-")
    data = yf.Ticker(symbol)
    tickerDf = data.history(period='1d', start='2024-05-30', end='2024-05-31')
    #print(symbol)
    marketCap= data.info['marketCap']
    price= tickerDf['Open'][0]
    full_df = full_df.append(
        pd.Series(
        [
            symbol,
            price,
            marketCap,
            'N/A'
        ],
        index=df_columns
        ),
        ignore_index=True
    )
    
full_df

,Ticker,Stock Price,Market Cap,# of shares to buy
0,MMM,97.690002,54566932480,N/A
1,AOS,81.260002,11916937216,N/A
2,ABT,101.279999,179929923584,N/A
3,ABBV,154.649994,286318166016,N/A
4,ACN,289.970001,181111685120,N/A
...,...,...,...,...
498,XYL,138.050003,33239486464,N/A
499,YUM,135.059998,39862190080,N/A
500,ZBRA,319.549988,15568250880,N/A
501,ZBH,115.459999,23337785344,N/A
